In [1]:
import SimpleITK as sitk
import os
import numpy as np

# 定义路径
labelsTr_nrrd_dir = "/Users/ziling/Desktop/MRCP/GT_NRRD/Radiopaedia"
imagesTr_nii_dir = "/Users/ziling/Desktop/MRCP/NII_img_Radiopaedia"
output_labelsTr_dir = "/Users/ziling/Desktop/MRCP/NII_GT_Radiopaedia"
os.makedirs(output_labelsTr_dir, exist_ok=True)

# labelsTs_nrrd_dir = r"D:\nnUNet\DATASET\Task022_MRCP\labelsTs"
# imagesTs_nii_dir = r"D:\nnUNet\DATASET\Task022_MRCP\imagesTs"
# output_labelsTs_dir = r"D:\nnUNet\DATASET\Task022_MRCP\labelsTs_converted"
# os.makedirs(output_labelsTs_dir, exist_ok=True)

def convert_nrrd_to_nii_gz(nrrd_dir, nii_dir, output_dir):
    nrrd_files = [f for f in os.listdir(nrrd_dir) if f.endswith('.nrrd') and not f.startswith('.')]
    for nrrd_file in nrrd_files:
        # 读取nrrd文件
        nrrd_path = os.path.join(nrrd_dir, nrrd_file)
        label_img = sitk.ReadImage(nrrd_path)
        label_arr = sitk.GetArrayFromImage(label_img)
        label_arr_trans=np.zeros((label_arr.shape))
        label_arr_trans[label_arr!=0] = 1
        label_img_tran = sitk.GetImageFromArray(label_arr_trans)

        # 对应的nii.gz图像文件
        nii_file = nrrd_file.replace('.nrrd', '.nii.gz')  # 假设nrrd文件和nii文件有相同的基础名称
        nii_path = os.path.join(nii_dir, nii_file)

        if not os.path.exists(nii_path):
            print(f"Warning: Corresponding NIfTI file {nii_file} not found for {nrrd_file}")
            continue

        # 读取nii.gz文件的三维信息
        ref_img = sitk.ReadImage(nii_path)

        # 检查尺寸是否匹配
        if label_img.GetSize() == ref_img.GetSize():
            # 调整label_img的方向、间隔和起始点与ref_img一致
            label_img_tran.CopyInformation(ref_img)

            # 保存为nii.gz格式
            output_path = os.path.join(output_dir, nrrd_file.replace('.nrrd', '.nii.gz'))
            sitk.WriteImage(label_img_tran, output_path)
            # print(f"Converted {nrrd_file} to {output_path}")
        else:
            print(f"Size mismatch: {nrrd_file} and {nii_file} have different sizes. Conversion skipped.")

# 转换labelsTr目录下的文件
convert_nrrd_to_nii_gz(labelsTr_nrrd_dir, imagesTr_nii_dir, output_labelsTr_dir)

# 转换labelsTs目录下的文件
# convert_nrrd_to_nii_gz(labelsTs_nrrd_dir, imagesTs_nii_dir, output_labelsTs_dir)


In [ ]:
import os
import shutil

# 定义路径
imagesTr_dir = r"D:\nnUNet\DATASET\Task022_MRCP\imagesTr"
labelsTr_dir = r"D:\nnUNet\DATASET\Task022_MRCP\labelsTr_converted"
imagesTs_dir = r"D:\nnUNet\DATASET\Task022_MRCP\imagesTs"
labelsTs_dir = r"D:\nnUNet\DATASET\Task022_MRCP\labelsTs_converted"

extra_imagesTr_dir = r"D:\nnUNet\DATASET\Task022_MRCP\extra_imagesTr"
extra_imagesTs_dir = r"D:\nnUNet\DATASET\Task022_MRCP\extra_imagesTs"

# 创建输出目录
os.makedirs(extra_imagesTr_dir, exist_ok=True)
os.makedirs(extra_imagesTs_dir, exist_ok=True)

def move_extra_files(image_dir, label_dir, output_dir):
    # 获取文件列表，不包括文件扩展名
    image_files = {f.split(".nii.gz")[0] for f in os.listdir(image_dir) if f.endswith('.nii.gz')}
    label_files = {f.split(".nii.gz")[0] for f in os.listdir(label_dir) if f.endswith('.nii.gz')}

    # 找出在image目录中但不在label目录中的文件
    extra_files = image_files - label_files

    for extra_file in extra_files:
        # 找到实际的文件路径并移动
        file_to_move = os.path.join(image_dir, f"{extra_file}.nii.gz")
        if os.path.exists(file_to_move):
            shutil.move(file_to_move, os.path.join(output_dir, f"{extra_file}.nii.gz"))
            print(f"Moved {extra_file}_0000.nii.gz to {output_dir}")
        else:
            print(f"File not found: {file_to_move}")

# 处理imagesTr目录
move_extra_files(imagesTr_dir, labelsTr_dir, extra_imagesTr_dir)

# 处理imagesTs目录
move_extra_files(imagesTs_dir, labelsTs_dir, extra_imagesTs_dir)